# Context
This notebook drives the training process for different models.

In [3]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../project.env")
sys.path.append(os.environ["PYTHONPATH"])

import pickle

In [4]:
# Import project-wide and PH2 specific variables and functions
import superheader as sup



Chosen class grouping: two-classes


Directory /Users/diego/Desktop/iteso/TOG/ exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/src exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/bin exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/media exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/scores exists. Continuing with execution


# Datasets

In [5]:
sup.report_dir_if_not_exists(sup.PH1_DATA_ROOT)
sup.report_dir_if_not_exists(sup.PH2_DATA_ROOT)
sup.report_dir_if_not_exists(sup.PH3_DATA_ROOT)

Directory /Users/diego/Desktop/iteso/TOG/data/PH1/two-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH2/two-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH3/two-classes exists. Continuing with execution


# BERT

In [8]:
from transformers import BertModel, BertConfig

bert = BertModel.from_pretrained("bert-base-uncased")

In [9]:
import torch.nn as nn

input_dim = 10 # for now
hidden_size = bert.config.hidden_size  # usually 768

projection = nn.Linear(input_dim, hidden_size)

In [ ]:
# x: (batch_size, seq_len, input_dim)
x_proj = projection(x)

# Optional: add a [CLS] token (learned)
cls_token = nn.Parameter(torch.zeros(1, 1, hidden_size))
cls = cls_token.expand(x_proj.size(0), -1, -1)
x_with_cls = torch.cat([cls, x_proj], dim=1)

# Create attention mask (1s for real tokens)
attention_mask = torch.ones(x_with_cls.size()[:2]).to(x.device)

# Forward through pretrained BERT
outputs = bert(inputs_embeds=x_with_cls, attention_mask=attention_mask)
pooled_output = outputs.last_hidden_state[:, 0]  # [CLS] token

In [ ]:
# Assume: x = [batch, seq_len, input_dim]
x_proj = projection(x)                            # shape: [B, T, H]
x_proj = torch.cat([cls_token.repeat(B, 1, 1), x_proj], dim=1)
mask = torch.ones(x_proj.size()[:2]).to(device)

outputs = bert(inputs_embeds=x_proj, attention_mask=mask)
cls_out = outputs.last_hidden_state[:, 0]
logits = classifier(cls_out)